In [5]:
from utils.models import random_forest, log_reg, knn
from sklearn.ensemble import VotingClassifier
from utils.plot import evaluation_metrics, evaluation_metric
from utils.model import get_cleaned_data, split_train_test, finetune_model
data = get_cleaned_data("./data/kickstarter_projects.csv")
xtrain,xtest,ytrain,ytest = split_train_test(data, "success")

In [1]:
models = [("knn",knn),("log_reg",log_reg), ("random_forest",random_forest)]
ensemble = finetune_model(
    VotingClassifier(estimators=models, voting="hard"),
    {
        #"knn__KNN__n_neighbors": [1,5,10],
        #"knn__KNN__p": [1,2,3],
        #"knn__KNN__weights": ["uniform", "distance"],
        #"knn__KNN__leaf_size": [20,30,40],
        #"knn__KNN__algorithm": ["ball_tree","kd_tree","brute"],
        "log_reg__LogisticRegression__C": [0.1 ,0.2, 0.5, 0.7, 0.9, 1],
        "log_reg__LogisticRegression__solver":["lbfgs", "liblinear", "newton-cg", "newton-cholesky", "sag", "saga"],
        "log_reg__LogisticRegression__max_iter":[500, 1000, 2000],
        'random_forest__random_forest__criterion':["gini", "entropy", "log_loss"],
        'random_forest__random_forest__max_depth': [5,10,15,20],
        'random_forest__random_forest__max_features':[ 'sqrt',"log2"],
        'random_forest__random_forest__max_leaf_nodes': [10, 25, 50],
        'random_forest__random_forest__min_samples_leaf': [2,5, 10, 15],
        'random_forest__random_forest__min_samples_split':  [2,5, 10, 15],
        'random_forest__random_forest__n_estimators':  [80,100,120]
    },
    xtrain,
    ytrain
)

NameError: name 'knn' is not defined